In [3]:
#source ~/.bashrc
#http://localhost:4040/executors/   後台
#pyspark --master local[*] --executor-memory 4G --driver-memory 2G --packages com.databricks:spark-csv_2.10:1.4.0
#(加起來不要超過8  --executor-memory 2g   --driver-memory 2g 處理排程)
#啟動pyspark指令：
#pyspark --packages com.databricks:spark-csv_2.10:1.4.0
#pyspark --packages com.databricks:spark-csv_2.10:1.4.0 --master local[*] --executor-memory 4G
from pyspark.sql import SQLContext
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.ml.evaluation import MulticlassClassificationEvaluator
from pyspark.sql.types import *
from pyspark.ml.feature import RFormula

from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.util import MLUtils

from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression

from pyspark.ml.tuning import *

In [4]:
from pyspark.sql import SQLContext
from pyspark.sql.types import *

# Load and parse the data file, converting it to a DataFrame.
data = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("/home/sheng/Downloads/Click_Through_Rate_Prediction/Click_Through_Rate_Prediction/train10000.csv")
#data = sqlContext.read.format("com.databricks.spark.csv").options(header='true', inferschema='true').load("/home/sheng/Downloads/Click_Through_Rate_Prediction/Click_Through_Rate_Prediction/train.csv")
# Displays the content of the DataFrame to stdout
#df.show()
#data.show()

In [5]:
# Print the schema in a tree format
data.printSchema()

root
 |-- id: decimal(20,0) (nullable = true)
 |-- click: integer (nullable = true)
 |-- hour: integer (nullable = true)
 |-- C1: integer (nullable = true)
 |-- banner_pos: integer (nullable = true)
 |-- site_id: string (nullable = true)
 |-- site_domain: string (nullable = true)
 |-- site_category: string (nullable = true)
 |-- app_id: string (nullable = true)
 |-- app_domain: string (nullable = true)
 |-- app_category: string (nullable = true)
 |-- device_id: string (nullable = true)
 |-- device_ip: string (nullable = true)
 |-- device_model: string (nullable = true)
 |-- device_type: integer (nullable = true)
 |-- device_conn_type: integer (nullable = true)
 |-- C14: integer (nullable = true)
 |-- C15: integer (nullable = true)
 |-- C16: integer (nullable = true)
 |-- C17: integer (nullable = true)
 |-- C18: integer (nullable = true)
 |-- C19: integer (nullable = true)
 |-- C20: integer (nullable = true)
 |-- C21: integer (nullable = true)



In [6]:
# Select only the "name" column
Schema = data.select("id","click","hour","C1","banner_pos","site_id","site_category").show()

+--------------------+-----+--------+----+----------+--------+-------------+
|                  id|click|    hour|  C1|banner_pos| site_id|site_category|
+--------------------+-----+--------+----+----------+--------+-------------+
| 1000009418151094273|    0|14102100|1005|         0|1fbe01fe|     28905ebd|
|10000169349117863715|    0|14102100|1005|         0|1fbe01fe|     28905ebd|
|10000371904215119486|    0|14102100|1005|         0|1fbe01fe|     28905ebd|
|10000640724480838376|    0|14102100|1005|         0|1fbe01fe|     28905ebd|
|10000679056417042096|    0|14102100|1005|         1|fe8cc448|     0569f928|
|10000720757801103869|    0|14102100|1005|         0|d6137915|     f028772b|
|10000724729988544911|    0|14102100|1005|         0|8fda644b|     f028772b|
|10000918755742328737|    0|14102100|1005|         1|e151e245|     f028772b|
|10000949271186029916|    1|14102100|1005|         0|1fbe01fe|     28905ebd|
|10001264480619467364|    0|14102100|1002|         0|84c7ba46|     50e219e0|

In [7]:
Schema = StructType([
      StructField("id", DoubleType(), True),
      StructField("click", DoubleType(), True),
      StructField("hour", StringType(), True),
      StructField("C1", DoubleType(), True),
      StructField("banner_pos", DoubleType(), True),
      StructField("site_id", StringType(), True),
      StructField("site_domain", StringType(), True),
      StructField("site_category", StringType(), True),
      StructField("app_id", StringType(), True),
      StructField("app_domain", StringType(), True),
      StructField("app_category", StringType(), True),
      StructField("device_id", StringType(), True),
      StructField("device_ip", StringType(), True),
      StructField("device_model", StringType(), True),
      #StructField("device_type", DoubleType(), True),
      #StructField("device_conn_type", DoubleType(), True),
      #StructField("C14", DoubleType(), True),
      #StructField("C15", DoubleType(), True),
      #StructField("C16", DoubleType(), True),
      #StructField("C17", DoubleType(), True),
      #StructField("C18", DoubleType(), True),
      #StructField("C19", DoubleType(), True),
      #StructField("C20", DoubleType(), True),
      #StructField("C21", DoubleType(), True)
        
      StructField("device_type", StringType(), True),
      StructField("device_conn_type", StringType(), True),
      StructField("C14", DoubleType(), True),
      StructField("C15", DoubleType(), True),
      StructField("C16", DoubleType(), True),
      StructField("C17", DoubleType(), True),
      StructField("C18", DoubleType(), True),
      StructField("C19", DoubleType(), True),
      StructField("C20", DoubleType(), True),
      StructField("C21", DoubleType(), True)
    ])

In [8]:
#data.printSchema() #Data type doesn't change???? 
print Schema

StructType(List(StructField(id,DoubleType,true),StructField(click,DoubleType,true),StructField(hour,StringType,true),StructField(C1,DoubleType,true),StructField(banner_pos,DoubleType,true),StructField(site_id,StringType,true),StructField(site_domain,StringType,true),StructField(site_category,StringType,true),StructField(app_id,StringType,true),StructField(app_domain,StringType,true),StructField(app_category,StringType,true),StructField(device_id,StringType,true),StructField(device_ip,StringType,true),StructField(device_model,StringType,true),StructField(device_type,StringType,true),StructField(device_conn_type,StringType,true),StructField(C14,DoubleType,true),StructField(C15,DoubleType,true),StructField(C16,DoubleType,true),StructField(C17,DoubleType,true),StructField(C18,DoubleType,true),StructField(C19,DoubleType,true),StructField(C20,DoubleType,true),StructField(C21,DoubleType,true)))


In [9]:
from pyspark.ml.feature import StringIndexer
## Index labels, adding metadata to the label column.
## Fit on whole dataset to include all labels in index.
data = StringIndexer(inputCol="click", outputCol="label").fit(data).transform(data)
data.show()
## 可產生另一個檔案.transform(data)不一定要在（data）檔案裡
#labelIndexer  ===> data

+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+-----+
|                  id|click|    hour|  C1|banner_pos| site_id|site_domain|site_category|  app_id|app_domain|app_category|device_id|device_ip|device_model|device_type|device_conn_type|  C14|C15|C16| C17|C18|C19|   C20|C21|label|
+--------------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+---+------+---+-----+
| 1000009418151094273|    0|14102100|1005|         0|1fbe01fe|   f3845767|     28905ebd|ecad2386|  7801e8d9|    07d7df22| a99f214a| ddd2926e|    44956a24|          1|               2|15706|320| 50|1722|  0| 35|    -1| 79|  0.0|
|10000169349117863715|    0|14102100|1005|         0|1fbe01fe|   f3845767|     28905ebd|

In [10]:
# RFormula
from pyspark.ml.feature import RFormula
## RFormula: string input colums will be one-hot encoded, and numeric columns will be cast to doubles.
##特徵值要被修正formula" "
formula = RFormula(
    formula="label ~ banner_pos + app_id + site_category + site_id + site_domain + device_type + device_conn_type",
    #formula="label ~ banner_pos + app_id + site_category + site_id + site_domain + C14 + C17 + C18 + C19 + C21", #0.707636
    #formula="label ~ banner_pos + site_id + site_domain + C14 + C17 + C21", #0.7
    featuresCol="features",
    labelCol="label")
formula_data = formula.fit(data).transform(data)
formula_data.select("features","label").show()

+--------+-----+
|features|label|
+--------+-----+
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [1.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [1.0]|  0.0|
|   [0.0]|  1.0|
|   [0.0]|  0.0|
|   [1.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [0.0]|  0.0|
|   [1.0]|  0.0|
|   [0.0]|  0.0|
+--------+-----+
only showing top 20 rows



In [11]:
# Split the data into training and test sets (30% held out for testing)
#已經有了！
# Split training and test data.
(training, test) = formula_data.randomSplit([0.7, 0.3], seed = 12345) #what's seed
training.show()


+-----------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+----+------+---+-----+--------+
|               id|click|    hour|  C1|banner_pos| site_id|site_domain|site_category|  app_id|app_domain|app_category|device_id|device_ip|device_model|device_type|device_conn_type|  C14|C15|C16| C17|C18| C19|   C20|C21|label|features|
+-----------------+-----+--------+----+----------+--------+-----------+-------------+--------+----------+------------+---------+---------+------------+-----------+----------------+-----+---+---+----+---+----+------+---+-----+--------+
|  168962721406625|    0|14102100|1005|         1|856e6d3f|   58a89a43|     f028772b|ecad2386|  7801e8d9|    07d7df22| a99f214a| c654b5ac|    f39b265e|          1|               0|19772|320| 50|2227|  0| 687|100081| 48|  0.0|   [1.0]|
|  182439553152007|    1|14102100|1005|         0|5b08c53b| 

In [12]:
# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and rf (random forest).
# Train a RandomForest model.
#rf = RandomForestClassifier(labelCol="label", featuresCol="features")

# Chain indexers and forest in a Pipeline
# stages=[labelIndexer, featureIndexer, rf]這部分在In[14]做完了所以只要處理rf
#pipeline = Pipeline(stages=[rf])

In [13]:
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params
from pyspark.ml.feature import HashingTF, Tokenizer
from pyspark.sql import Row
from pyspark.ml import Pipeline
from pyspark.ml.classification import RandomForestClassifier
from pyspark.ml.feature import StringIndexer, VectorIndexer
from pyspark.mllib.classification import LogisticRegressionWithLBFGS
from pyspark.ml.evaluation import BinaryClassificationEvaluator
from pyspark.mllib.util import MLUtils
#===========#what does it mean from here
# Configure an ML pipeline, which consists of three stages: tokenizer, hashingTF, and rf (random forest).
#rf = RandomForestClassifier().setMaxBins(70)
rf = RandomForestClassifier(numTrees=100, maxDepth=20, labelCol="label") #maxDepth=20, maxBins=64, 

pipeline = Pipeline(stages=[rf])
pipelineModel = pipeline.fit(training)
trainingPredictions = pipelineModel.transform(training)
#trainingPredictions.show()
trainingPredictions.select("prediction", "label", "features").show()
testPredictions = pipelineModel.transform(test)

+----------+-----+--------+
|prediction|label|features|
+----------+-----+--------+
|       0.0|  0.0|   [1.0]|
|       0.0|  1.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [1.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  1.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  0.0|   [0.0]|
|       0.0|  1.0|   [0.0]|
+----------+-----+--------+
only showing top 20 rows



In [14]:
#evaluator = MulticlassClassificationEvaluator(
#labelCol="label", predictionCol="prediction", metricName="precision")
evaluator = BinaryClassificationEvaluator()

In [ ]:
from pyspark.mllib.linalg import Vectors
from pyspark.ml.classification import LogisticRegression
from pyspark.ml.param import Param, Params

evaluatorParaMap = {evaluator.metricName: "areaUnderROC"}
aucTraining = evaluator.evaluate(trainingPredictions, evaluatorParaMap)
aucTest = evaluator.evaluate(testPredictions, evaluatorParaMap)

print("pipeline Test AUC: %g" % aucTest)

pipeline Test AUC: 0.515099


In [ ]:
#Cross validation
from pyspark.ml.tuning import *
# The multiplies out to (2 x 3 x 3) x 10 = 180 different models being trained.
# k = 3 and k = 10 are common
#from pyspark.ml.tuning import *
#paramGrid = ParamGridBuilder().addGrid(rf.impurity, ['entropy', 'gini']).addGrid(rf.numTrees, [30, 50, 100]).build() #[10, 50, 100]高 50
paramGrid = ParamGridBuilder().addGrid(rf.maxDepth, [10,20,30]).addGrid(rf.impurity, ['entropy', 'gini']).addGrid(rf.numTrees, [30, 50, 100]).build()
#(rf.maxDepth, [10,20,30])
#println(paramGrid(1))

#=============#以上未做cv 以下做cv
cv = CrossValidator().setEstimator(pipeline).setEvaluator(evaluator).setEstimatorParamMaps(paramGrid).setNumFolds(3) #setNumFolds(3)
# Run cross-validation, and choose the best set of parameters.
cvModel = cv.fit(training)
cvPredictions = cvModel.transform(test)
cvAUCTest = evaluator.evaluate(cvPredictions, evaluatorParaMap)
cvPredictions.show()

#println("pipeline Training AUC: " + aucTraining)
print("pipeline Test AUC: %g" % aucTest)
print("Cross-Validation test AUC: %g" % cvAUCTest)